In [1]:
import numpy as np
import random
import random

In [2]:
nums = [75, 3, 1, 4, 50, 6]
operadores = {1:"+", 2:"-", 3:"*"}

In [3]:
class DNA():
    def __init__(self, n_indiv, n_gener, target, crossover_rate, mutation_rate, n_selection, selection_type, crossover_type):
        self.n_indiv = n_indiv
        self.n_gener = n_gener
        self.target = target
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.n_selection = n_selection
        self.selection_type = selection_type
        self.crossover_type = crossover_type
        self.best_fitness = [10000, [], 0] # fitness_value, number of iters without update
        self.lista_best_fitness = []

    def create_individuo(self, min=1, max=4):
        individuo = list(np.random.randint(min,max,len(nums)-1))
        return individuo
    
    def create_population(self):
        population = [self.create_individuo() for i in range(self.n_indiv)]
        return population
    
    def fitness(self, individuo):
        value = nums[0]
        for i, elem in enumerate(individuo):
            if elem == 1:
                value += nums[i+1]
            elif elem == 2:
                value -= nums[i+1]
            elif elem == 3:
                value *= nums[i+1]
        if value < 0:
            value = abs(value - self.target)
        else:
            value = abs(self.target - value)
        return value
    
    def evaluate_generation(self, population):
        scores = [self.fitness(individuo) for individuo in population]
        if min(scores) < self.best_fitness[0]:
            self.best_fitness[0] = min(scores)
            self.best_fitness[1] = population[scores.index(min(scores))]
            self.best_fitness[2] = 0
        else:
            self.best_fitness[2] += 1
        return 
        
    def k_mejores(self, population):
        scores = [(individuo, self.fitness(individuo)) for individuo in population]
        individuals_sorted = [elem[0] for elem in sorted(scores, key=lambda tup: tup[1])]
        return individuals_sorted[:self.n_selection]
        
    def tournament(self, population):
        new_population = []
        scores = [[individuo, self.fitness(individuo), 0] for individuo in population]
        while len(new_population) < self.n_indiv:
            P1 = np.random.randint(0, len(scores))
            P2 = np.random.randint(0, len(scores))
            while P1 == P2:
                P2 = np.random.randint(0, len(scores))
            parent1 = scores[P1]
            parent2 = scores[P2]
            scores[P1][2] += 1
            scores[P2][2] += 1
            winner = min([parent1, parent2], key = lambda t: t[1])
            new_population.append(winner[0])
            if scores[P1] == 2:
                scores.pop(P1)
            if scores[P2] == 2:
                scores.pop(P2)
        return new_population
    
    def selection(self, population):
        if self.selection_type == "k_mejores":
            population = self.k_mejores(population)
        else:
            population = self.tournament(population)
        return population
    
    def crossover_one_point(self,parents,new_population):
        index = np.random.choice(range(len(nums)-1))
        new_population.append([*parents[0][:index],*parents[1][index:]])
        new_population.append([*parents[1][:index],*parents[0][index:]])
        return new_population
    
    def crossover_two_points(self,parents,new_population):
        index1 = np.random.choice(range(len(nums)-1))
        index2 = np.random.choice(range(len(nums)-1))
        while index1 == index2:
            index2 = np.random.choice(range(len(nums)-1))
        sorted_index = sorted([index1,index2])
    
        new_population.append([*parents[0][:sorted_index[0]],*parents[1][sorted_index[0]:sorted_index[1]],*parents[0][sorted_index[1]:]])
        new_population.append([*parents[1][:sorted_index[0]],*parents[0][sorted_index[0]:sorted_index[1]],*parents[1][sorted_index[1]:]])
        return new_population
        
    def crossover_uniform(self,parents,new_population):
        mask = random.choices([0,1], k=len(nums)-1)
        children1 = []
        children2 = []
        for index, value in enumerate(mask):
            if value == 1:
                children1.append(parents[0][index])
                children2.append(parents[1][index])
            else:
                children1.append(parents[1][index])
                children2.append(parents[0][index])
        new_population.append(children1)
        new_population.append(children2)
        return new_population
        
    def reproduction(self, candidates):
        new_population = []
        while len(new_population) < self.n_indiv:
            parents = random.sample(candidates, 2)
            if random.random() <= self.crossover_rate:
                if self.crossover_type == "one-point":
                    new_population = self.crossover_one_point(parents, new_population)
                elif self.crossover_type == "two-points":
                    new_population = self.crossover_two_points(parents, new_population)
                else:
                    new_population = self.crossover_uniform(parents, new_population)
            else:
                new_population.append(parents[0])
                new_population.append(parents[1])
                candidates.remove(parents[0])
                candidates.remove(parents[1])
        return new_population
    
    def mutation(self, population):
        for i, indv in enumerate(population):
            for j in range(len(indv)):
                if random.random() <= self.mutation_rate:
                    new_alelo = np.random.randint(1, 4)
                    while new_alelo == indv[j]:
                        new_alelo = np.random.randint(1, 4)
                    population[i][j] = new_alelo
                
        return population
    
    def run(self):
        population = self.create_population()
        for i in range(self.n_gener):
            self.evaluate_generation(population)
            if self.best_fitness[0] != 0 and self.best_fitness[2] < 15:
                selected = self.selection(population)
                population = self.reproduction(selected)
                population = self.mutation(population)
            else:
                break
        return self.best_fitness[0], self.best_fitness[1]

### EJECUCIÓN FINAL DEL PROBLEMA

In [5]:
error_optimo = 10000
solucion_optima = []
number_iterations = 100
combinations = [("one-point",0.4,0.1),("two-points",0.3,0.1),("uniform",0.7,0.3)]
params = {"n_indiv":10, "n_gener":100, "target":852, "crossover_rate":combinations[1][1],
                    "mutation_rate":combinations[2][2], "n_selection":20, "selection_type":"tournament",
                    "crossover_type":combinations[2][0]}
for i in range(number_iterations):
    model = DNA(params["n_indiv"], params["n_gener"], params["target"], params["crossover_rate"], params["mutation_rate"], params["n_selection"]
        , params["selection_type"], params["crossover_type"])
    error, indv = model.run()
    if error < error_optimo:
        error_optimo = error
        solucion = [operadores[elem] for elem in indv]
        solucion_optima = solucion
print(error_optimo)
print(solucion_optima)

0
['*', '-', '*', '-', '+']
